# **Sai Ram**

### *https://www.kaggle.com/c/home-credit-default-risk*

In [365]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
import seaborn as sns
from sklearn import metrics, preprocessing
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from datetime import datetime, timedelta

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
pd.set_option('display.max_columns', 500)

## Consolidated Helper Functions

In [2]:
import shutil
import requests
import base64

# Encode text values to dummy variables(i.e. [1,0,0],[0,1,0],[0,0,1] for red,green,blue)
def encode_text_dummy(df, name):
    dummies = pd.get_dummies(df[name])
    for x in dummies.columns:
        dummy_name = "{}-{}".format(name, x)
        df[dummy_name] = dummies[x]
    df.drop(name, axis=1, inplace=True)


# Encode text values to a single dummy variable.  The new columns (which do not replace the old) will have a 1
# at every location where the original column (name) matches each of the target_values.  One column is added for
# each target value.
def encode_text_single_dummy(df, name, target_values):
    for tv in target_values:
        l = list(df[name].astype(str))
        l = [1 if str(x) == str(tv) else 0 for x in l]
        name2 = "{}-{}".format(name, tv)
        df[name2] = l


# Encode text values to indexes(i.e. [1],[2],[3] for red,green,blue).
def encode_text_index(df, name):
    le = preprocessing.LabelEncoder()
    df[name] = le.fit_transform(df[name])
    return le.classes_


# Encode a numeric column as zscores
def encode_numeric_zscore(df, name, mean=None, sd=None):
    if mean is None:
        mean = df[name].mean()

    if sd is None:
        sd = df[name].std()

    df[name] = (df[name] - mean) / sd


# Convert all missing values in the specified column to the median
def missing_median(df, name):
    med = df[name].median()
    df[name] = df[name].fillna(med)


# Convert all missing values in the specified column to the default
def missing_default(df, name, default_value):
    df[name] = df[name].fillna(default_value)


# Convert a Pandas dataframe to the x,y inputs that TensorFlow needs
def to_xy(df, target):
    result = []
    for x in df.columns:
        if x != target:
            result.append(x)
    # find out the type of the target column.  Is it really this hard? :(
    target_type = df[target].dtypes
    target_type = target_type[0] if hasattr(target_type, '__iter__') else target_type
    # Encode to int for classification, float otherwise. TensorFlow likes 32 bits.
    if target_type in (np.int64, np.int32):
        # Classification
        dummies = pd.get_dummies(df[target])
        return df.as_matrix(result).astype(np.float32), dummies.as_matrix().astype(np.float32)
    else:
        # Regression
        return df.as_matrix(result).astype(np.float32), df.as_matrix([target]).astype(np.float32)

# Nicely formatted time string
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)


# Regression chart.
def chart_regression(pred,y,sort=True):
    t = pd.DataFrame({'pred' : pred, 'y' : y.flatten()})
    if sort:
        t.sort_values(by=['y'],inplace=True)
    a = plt.plot(t['y'].tolist(),label='expected')
    b = plt.plot(t['pred'].tolist(),label='prediction')
    plt.ylabel('output')
    plt.legend()
    plt.show()

# Remove all rows where the specified column is +/- sd standard deviations
def remove_outliers(df, name, sd):
    drop_rows = df.index[(np.abs(df[name] - df[name].mean()) >= (sd * df[name].std()))]
    df.drop(drop_rows, axis=0, inplace=True)


# Encode a column to a range between normalized_low and normalized_high.
def encode_numeric_range(df, name, normalized_low=-1, normalized_high=1,
                         data_low=None, data_high=None):
    if data_low is None:
        data_low = min(df[name])
        data_high = max(df[name])

    df[name] = ((df[name] - data_low) / (data_high - data_low)) \
               * (normalized_high - normalized_low) + normalized_low
        
# This function submits an assignment.  You can submit an assignment as much as you like, only the final
# submission counts.  The paramaters are as follows:
# data - Pandas dataframe output.
# key - Your student key that was emailed to you.
# no - The assignment class number, should be 1 through 1.
# source_file - The full path to your Python or IPYNB file.  This must have "_class1" as part of its name.  
# .             The number must match your assignment number.  For example "_class2" for class assignment #2.
def submit(data,key,no,source_file=None):
    if source_file is None and '__file__' not in globals(): raise Exception('Must specify a filename when a Jupyter notebook.')
    if source_file is None: source_file = __file__
    suffix = '_class{}'.format(no)
    if suffix not in source_file: raise Exception('{} must be part of the filename.'.format(suffix))
    with open(source_file, "rb") as image_file:
        encoded_python = base64.b64encode(image_file.read()).decode('ascii')
    ext = os.path.splitext(source_file)[-1].lower()
    if ext not in ['.ipynb','.py']: raise Exception("Source file is {} must be .py or .ipynb".format(ext))
    r = requests.post("https://api.heatonresearch.com/assignment-submit",
        headers={'x-api-key':key}, json={'csv':base64.b64encode(data.to_csv(index=False).encode('ascii')).decode("ascii"),
        'assignment': no, 'ext':ext, 'py':encoded_python})
    if r.status_code == 200:
        print("Success: {}".format(r.text))
    else: print("Failure: {}".format(r.text))

## Visualization Functions

In [3]:
# Plot a confusion matrix.
# cm is the confusion matrix, names are the names of the classes.
def plot_confusion_matrix(cm, names, title='Confusion matrix', cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(names))
    plt.xticks(tick_marks, names, rotation=45)
    plt.yticks(tick_marks, names)
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    

# Plot an ROC. pred - the predictions, y - the expected output.
def plot_roc(pred,y):
    fpr, tpr, _ = roc_curve(y, pred)
    roc_auc = auc(fpr, tpr)

    plt.figure()
    plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC)')
    plt.legend(loc="lower right")
    plt.show()

## Set the desired TensorFlow output level for this example

In [4]:
tf.logging.set_verbosity(tf.logging.ERROR)

## Set filepath + filename

In [5]:
!ls

HomeCreditDefaultRisk.ipynb        bureau_balance.csv
HomeCredit_columns_description.csv bureau_balance_agg.hdf
POS_CASH_balance.csv               credit_card_balance.csv
application_test.csv               installments_payments.csv
application_train.csv              previous_application.csv
bureau.csv


Read CSVs

In [6]:
path = !pwd
# filename = os.path.join(path,"application_test.csv")

In [7]:
# bureau_balance = pd.read_csv('bureau_balance.csv', na_values=['NA','?'], sep=',',index_col='SK_ID_BUREAU')

In [8]:
# bureau_balance_past_tenure = pd.DataFrame(bureau_balance.groupby('SK_ID_BUREAU')['MONTHS_BALANCE'].min())

In [9]:
# bureau_balance_past_status = pd.DataFrame(bureau_balance.groupby('SK_ID_BUREAU')['STATUS'].value_counts())
# bureau_balance_past_status.columns = ['STATUS_COUNT']
# bureau_balance_past_status.reset_index(inplace=True)

In [10]:
# bureau_balance_past_status = bureau_balance_past_status.pivot_table(index=['SK_ID_BUREAU'], 
#             columns=['STATUS'], values='STATUS_COUNT').fillna(0)

In [11]:
# bureau_balance_agg = pd.merge(bureau_balance_past_status, bureau_balance_past_tenure, left_index = True, right_index = True)

In [12]:
# bureau_balance_agg.to_hdf('bureau_balance_agg.hdf','mydata',mode='w')

In [142]:
# bureau_balance = pd.read_hdf('bureau_balance_agg.hdf','mydata')

In [211]:
# bureau_balance.shape

In [212]:
# bureau = pd.read_csv('bureau.csv', na_values=['NA','?'], sep=',',index_col='SK_ID_CURR')

In [213]:
# bureau.reset_index(inplace=True)

In [214]:
# bureau.set_index('SK_ID_BUREAU',inplace=True)

In [215]:
# bureau = bureau.join(bureau_balance)

In [216]:
# CREDIT_ACTIVE - Pivot
# CREDIT_TYPE - Pivot
# CREDIT_CURRENCY - Pivot

# Numerics:
# DAYS_CREDIT - max, min, mean
# CREDIT_DAY_OVERDUE - max, min, mean
# DAYS_CREDIT_ENDDATE - max, min, mean
# DAYS_ENDDATE_FACT - 

In [217]:
# bureau._get_numeric_data().columns

In [218]:
# bureau.reset_index(inplace=True)

In [219]:
# buroh = bureau.groupby('SK_ID_CURR')['DAYS_CREDIT', 'CREDIT_DAY_OVERDUE',
#        'DAYS_CREDIT_ENDDATE', 'DAYS_ENDDATE_FACT', 'AMT_CREDIT_MAX_OVERDUE',
#        'CNT_CREDIT_PROLONG', 'AMT_CREDIT_SUM', 'AMT_CREDIT_SUM_DEBT',
#        'AMT_CREDIT_SUM_LIMIT', 'AMT_CREDIT_SUM_OVERDUE', 'DAYS_CREDIT_UPDATE',
#        'AMT_ANNUITY', '0', '1', '2', '3', '4', '5', 'C', 'X',
#        'MONTHS_BALANCE'].agg([pd.np.min, pd.np.max, pd.np.mean, pd.np.median])

In [220]:
# buroh['SK_ID_BUREAU_COUNT'] = bureau.groupby('SK_ID_CURR')['SK_ID_BUREAU'].count()

In [221]:
# CREDIT_ACTIVE - Pivot
# CREDIT_TYPE - Pivot
# CREDIT_CURRENCY - Pivot

In [222]:
# burohCats = pd.crosstab(bureau.SK_ID_CURR, bureau.CREDIT_ACTIVE).join(
#     pd.crosstab(bureau.SK_ID_CURR, bureau.CREDIT_TYPE)).join(
#     pd.crosstab(bureau.SK_ID_CURR, bureau.CREDIT_CURRENCY))

In [223]:
# buroh = buroh.join(burohCats)

In [224]:
# buroh.shape

In [225]:
# buroh.to_hdf('buroh.hdf','mydata',mode='w')

In [226]:
buroh = pd.read_hdf('buroh.hdf','mydata')

In [231]:
buroh.dtypes.value_counts()

float64    76
int64      32
dtype: int64

![title](Schema.png)

In [233]:
POS_CASH_balance = pd.read_csv('POS_CASH_balance.csv', na_values=['NA','?'], sep=',',index_col='SK_ID_CURR')

/Users/princeh/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [393]:
POS_CASH_balance.loc[POS_CASH_balance.SK_ID_PREV == 1004391].sort_values(['SK_ID_PREV','MONTHS_BALANCE'])

,SK_ID_PREV,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
SK_ID_CURR,,,,,,,
173320,1004391,-19,12.0,12.0,Active,0,0
173320,1004391,-18,12.0,11.0,Active,0,0
173320,1004391,-17,12.0,10.0,Active,0,0
173320,1004391,-16,12.0,8.0,Active,0,0
173320,1004391,-15,12.0,8.0,Active,0,0
173320,1004391,-14,12.0,7.0,Active,0,0
173320,1004391,-13,12.0,6.0,Active,0,0
173320,1004391,-12,12.0,4.0,Active,0,0
173320,1004391,-11,12.0,3.0,Active,0,0


In [ ]:
SK_ID_PREV.value_counts() -> count of prev loans
# min(MONTHS_BALANCE) -> agreed tenure
# max(MONTHS_BALANCE) -> advance settlement
/ min(MONTHS_BALANCE) - max(MONTHS_BALANCE) -> settlement tenure
CNT_INSTALMENT -> count of ups and count of downs and difference
POS_CASH_balance.CNT_INSTALMENT_FUTURE - POS_CASH_balance.CNT_INSTALMENT_FUTURE.shift(-1) -> 
NAME_CONTRACT_STATUS -> final status / POS_CASH_balance.groupby('SK_ID_PREV')['CNT_INSTALMENT_FUTURE'].min()
SK_DPD.sum() / SK_DPD_DEF.sum()

In [383]:
POS = pd.DataFrame(POS_CASH_balance.groupby(POS_CASH_balance.index)['SK_ID_PREV'].nunique())

In [391]:
POS_CASH_balance.SK_ID_PREV.nunique() == len(POS_CASH_balance)

10001358

In [387]:
POS_CASH_balance.groupby('SK_ID_PREV')['MONTHS_BALANCE'].agg([pd.np.min, pd.np.max, pd.np.mean, pd.np.median])

,amin,amax,mean,median
SK_ID_PREV,,,,
1000001,-10,-8,-9.0,-9.0
1000002,-54,-50,-52.0,-52.0
1000003,-4,-1,-2.5,-2.5
1000004,-29,-22,-25.5,-25.5
1000005,-56,-46,-51.0,-51.0
1000007,-5,-1,-3.0,-3.0
1000008,-43,-34,-38.5,-38.5
1000009,-16,-10,-13.0,-13.0
1000010,-19,-9,-14.0,-14.0


In [384]:
POS_CASH_balance.

,SK_ID_PREV,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
SK_ID_CURR,,,,,,,
182943,1803195,-31,48.0,45.0,Active,0,0
367990,1715348,-33,36.0,35.0,Active,0,0
397406,1784872,-32,12.0,9.0,Active,0,0
269225,1903291,-35,48.0,42.0,Active,0,0
334279,2341044,-35,36.0,35.0,Active,0,0
342166,2207092,-32,12.0,12.0,Active,0,0
204376,1110516,-38,48.0,43.0,Active,0,0
153211,1387235,-35,36.0,36.0,Active,0,0
112740,1220500,-31,12.0,12.0,Active,0,0


In [352]:
credit_card_balance = pd.read_csv('credit_card_balance.csv', na_values=['NA','?'], sep=',',index_col='SK_ID_CURR')

/Users/princeh/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [364]:
credit_card_balance.sort_values(['SK_ID_PREV','MONTHS_BALANCE'])

,SK_ID_PREV,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_CURRENT,AMT_PAYMENT_TOTAL_CURRENT,AMT_RECEIVABLE_PRINCIPAL,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,,
394447,1000018,-6,38879.145,45000,13500.0,51042.645,0.0,37542.645,0.000,9000.000,9000.000,37542.645,37542.645,37542.645,3.0,15,0.0,12.0,0.0,Active,0,0
394447,1000018,-5,40934.070,45000,0.0,2335.500,0.0,2335.500,2250.000,3900.870,3900.870,39576.780,40934.070,40934.070,0.0,2,0.0,2.0,1.0,Active,0,0
394447,1000018,-4,44360.505,45000,0.0,2032.560,0.0,2032.560,2250.000,3646.710,3646.710,43376.760,44360.505,44360.505,0.0,2,0.0,2.0,2.0,Active,0,0
394447,1000018,-3,113862.285,135000,13500.0,69156.945,0.0,55656.945,2263.770,3190.635,3190.635,108091.800,109150.785,109150.785,3.0,14,0.0,11.0,3.0,Active,0,0
394447,1000018,-2,136695.420,135000,0.0,22827.330,0.0,22827.330,6206.670,7970.535,7970.535,132903.000,136024.920,136024.920,0.0,11,0.0,11.0,4.0,Active,0,0
361282,1000030,-8,0.000,45000,NaN,0.000,NaN,NaN,0.000,NaN,0.000,0.000,0.000,0.000,NaN,0,NaN,NaN,0.0,Active,0,0
361282,1000030,-7,15583.635,45000,4500.0,31105.755,0.0,26605.755,0.000,16067.250,16067.250,15130.755,15138.135,15138.135,1.0,14,0.0,13.0,0.0,Active,0,0
361282,1000030,-6,33784.740,45000,0.0,20212.650,0.0,20212.650,0.000,2567.655,317.655,33725.745,33784.740,33784.740,0.0,8,0.0,8.0,0.0,Active,0,0
361282,1000030,-5,36885.285,45000,0.0,6368.850,0.0,6368.850,2250.000,5022.765,2772.765,36499.770,36885.285,36885.285,0.0,3,0.0,3.0,1.0,Active,0,0


In [332]:
# POS_CASH_balance.loc[(POS_CASH_balance.SK_DPD!=0) &(POS_CASH_balance.SK_DPD_DEF!=0)].SK_ID_PREV.value_counts()[POS_CASH_balance.loc[(POS_CASH_balance.SK_DPD!=0) &(POS_CASH_balance.SK_DPD_DEF!=0)].SK_ID_PREV.value_counts() == 2]

In [280]:
POS_CASH_balance.NAME_CONTRACT_STATUS.value_counts()

Active                   9151119
Completed                 744883
Signed                     87260
Demand                      7065
Returned to the store       5461
Approved                    4917
Amortized debt               636
Canceled                      15
XNA                            2
Name: NAME_CONTRACT_STATUS, dtype: int64

In [ ]:
installments_payments = pd.read_csv('installments_payments.csv', na_values=['NA','?'], sep=',',index_col='SK_ID_CURR')

In [349]:
previous_application = pd.read_csv('previous_application.csv', na_values=['NA','?'], sep=',',index_col='SK_ID_CURR')

/Users/princeh/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [ ]:
TrainDF = pd.read_csv('application_train.csv', na_values=['NA','?'], sep=',',index_col='SK_ID_CURR')

In [ ]:
TestDF = pd.read_csv('application_test.csv', na_values=['NA','?'], sep=',',index_col='SK_ID_CURR')

In [ ]:
# chunksize = 100000
# TextFileReader = pd.read_csv(filename,na_values=['NA','?'], engine='python', low_memory = True, sep=',', chunksize=chunksize, iterator=True)
# df = pd.concat(TextFileReader, ignore_index=True)

Converting Date Cols as integers

In [ ]:
# df['click_time'] =  pd.to_datetime(df.click_time).astype(np.int64)
# df['attributed_time'] =  pd.to_datetime(df.attributed_time).astype(np.int64)

# # Sort by date
# df = df.sort_values(by =['app','click_time'])

In [ ]:
# df.to_hdf('talkingData.hdf','mydata',mode='w')

In [ ]:
# df = pd.read_hdf('talkingData.hdf','mydata')

In [ ]:
# df.dtypes

In [ ]:
def mem_usage(pandas_obj):
    if isinstance(pandas_obj,pd.DataFrame):
        usage_b = pandas_obj.memory_usage(deep=True).sum()
    else: # we assume if not a df it's a series
        usage_b = pandas_obj.memory_usage(deep=True)
    usage_mb = usage_b / 1024 ** 2 # convert bytes to megabytes
    return "{:03.2f} MB".format(usage_mb)

In [ ]:
# df_int = df.select_dtypes(include = ['int'])
# converted_int = df_int.apply(pd.to_numeric, downcast = 'unsigned')

In [ ]:
# print(mem_usage(df_int))
# print(mem_usage(converted_int))

In [ ]:
# converted_int.to_hdf('converted_int.hdf','mydata',mode='w')

In [ ]:
# df = pd.read_hdf('converted_int.hdf','mydata')

In [ ]:
# mem_usage(df)

In [ ]:
# df['pace'] = np.where((df['app'] == df['app'].shift(1)), df['click_time'] - df['click_time'].shift(1), -1)

# df['ipChanged'] = np.where(((df['ip'] == df['ip'].shift(1)) & (df['app'] == df['app'].shift(1))), 0, 1)
# df['deviceChanged'] = np.where(((df['device'] == df['device'].shift(1)) & (df['app'] == df['app'].shift(1))), 0, 1)
# df['osChanged'] = np.where(((df['os'] == df['os'].shift(1)) & (df['app'] == df['app'].shift(1))), 0, 1)
# df['channelChanged'] = np.where(((df['channel'] == df['channel'].shift(1)) & (df['app'] == df['app'].shift(1))), 0, 1)

In [ ]:
# mem_usage(converted_int)

In [ ]:
# df_int = df.select_dtypes(include = ['int'])
# converted_int = df_int.apply(pd.to_numeric, downcast = 'unsigned')

In [ ]:
# df_float = df.select_dtypes(include=['float'])
# converted_float = df_float.apply(pd.to_numeric,downcast='float')

In [ ]:
# df1 = df.drop(columns=['attributed_time','pace','ipChanged','deviceChanged','osChanged','channelChanged'])

In [ ]:
# df1.to_hdf('df1.hdf','mydata',mode='w')

In [ ]:
# drew = pd.merge(df1, converted_int, left_index=True, right_index=True)

In [ ]:
# drew.to_hdf('drew.hdf','mydata',mode='w')

In [ ]:
# draft = pd.merge(drew, converted_float, left_index=True, right_index=True)

In [ ]:
# draft.drop(columns = 'pace', inplace = True)

In [ ]:
# draft.to_hdf('draft.hdf','mydata',mode='w')

In [ ]:
df = pd.read_hdf('draft.hdf','mydata')

In [ ]:
mem_usage(df)

In [ ]:
df = df.drop(columns = ['attributed_time'])

In [ ]:
# for col in ['is_attributed','ipChanged','deviceChanged','osChanged','channelChanged']:
#     df[col] = df[col].astype('category')

In [ ]:
mem_usage(df)

In [ ]:
# df.reset_index(inplace=True)
# df.drop(columns='index',axis=0,inplace=True)
# df.index.name = 'event'

In [ ]:
df['ipCount'] = 0
df['ipCount'] = df['ipCount'].apply(pd.to_numeric, downcast = 'unsigned')
df['deviceCount'] = 0
df['deviceCount'] = df['deviceCount'].apply(pd.to_numeric, downcast = 'unsigned')
df['osCount'] = 0
df['osCount'] = df['osCount'].apply(pd.to_numeric, downcast = 'unsigned')
df['channelCount'] = 0
df['channelCount'] = df['channelCount'].apply(pd.to_numeric, downcast = 'unsigned')
df['linkages'] = 0
df['linkages'] = df['linkages'].apply(pd.to_numeric, downcast = 'unsigned')

In [ ]:
df.to_hdf('df.hdf','mydata',mode='w')

In [ ]:
df = pd.read_hdf('df.hdf','mydata')

In [ ]:
# for col in ['ipCount','deviceCount','osCount','channelCount','linkages']:
#     df[col] = df[col].apply(pd.to_numeric, downcast = 'unsigned')

In [ ]:
ipCount = []
deviceCount = []
osCount = []
channelCount = []
linkages = []

for index, row in df.iterrows():
    ipCount.append(df.loc[((df.index <= index) & (df.app == row.app)),'ip'].nunique()).apply(pd.to_numeric, downcast = 'unsigned')
    deviceCount.append(df.loc[((df.index <= index) & (df.app == row.app)),'device'].nunique()).apply(pd.to_numeric, downcast = 'unsigned')
    osCount.append(df.loc[((df.index <= index) & (df.app == row.app)),'os'].nunique()).apply(pd.to_numeric, downcast = 'unsigned')
    channelCount.append(df.loc[((df.index <= index) & (df.app == row.app)),'channel'].nunique()).apply(pd.to_numeric, downcast = 'unsigned')
    linkages.append(df.loc[((df.index <= index) & (df.ip == row.ip)),'app'].nunique()).apply(pd.to_numeric, downcast = 'unsigned')
#     print(index, row.app)

In [ ]:
df.ipCount = ipCount
df.deviceCount = deviceCount
df.osCount = osCount
df.channelCount = channelCount
df.linkages = linkages

In [ ]:
# df_ip = pd.DataFrame(df.groupby('ip')['app'].nunique())
# df_ip.columns = ['linkages']
# df_ip_links = pd.merge(df, df_ip, left_on='ip', right_index=True)
# df_consolidated = pd.merge(df_ip_links, df, left_index=True, right_index=True)

In [ ]:
df = df.drop(columns = ['ip','device','app','channel','os','click_time','attributed_time'], axis=1)

In [ ]:
# df.ip = df.ip.astype(str)
# df.app = df.app.astype(str)
# df.device = df.device.astype(str)
# df.os = df.os.astype(str)
# df.channel = df.channel.astype(str)

In [ ]:
df.head()

In [ ]:
# Encode feature vector
# df.drop(['ip', 'device', 'os', 'channel', 'click_time'],axis=1,inplace=True)
diagnosis = encode_text_index(df,'is_attributed')
num_classes = len(diagnosis)

In [ ]:
# Create the x-side (feature vectors) of the training
x, y = to_xy(df,'is_attributed')

In [ ]:
df

In [ ]:
# Split into train/test
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)

In [ ]:
# Build network
model = Sequential()
model.add(Dense(50, input_dim=x.shape[1], activation='relu'))
model.add(Dense(25, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(y.shape[1],activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=1, mode='auto')
checkpointer = ModelCheckpoint(filepath="best_weights.hdf5", verbose=0, save_best_only=True) # save best model
model.fit(x_train,y_train,validation_data=(x_test,y_test),callbacks=[monitor,checkpointer],verbose=0,epochs=1000)
model.load_weights('best_weights.hdf5') # load weights from best model

In [ ]:
# Measure accuracy
pred = model.predict(x_test)
pred = np.argmax(pred,axis=1)
y_compare = np.argmax(y_test,axis=1)
score = metrics.accuracy_score(y_compare, pred)
print("Final accuracy: {}".format(score))

In [ ]:
# !conda install keras

Confusion Matrix

In [ ]:
from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

# Compute confusion matrix
cm = confusion_matrix(y_compare, pred)
np.set_printoptions(precision=2)
print('Confusion matrix, without normalization')
print(cm)
plt.figure()
plot_confusion_matrix(cm, diagnosis)

# Normalize the confusion matrix by row (i.e by the number of samples
# in each class)
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
print('Normalized confusion matrix')
print(cm_normalized)
plt.figure()
plot_confusion_matrix(cm_normalized, diagnosis, title='Normalized confusion matrix')

plt.show()

In [ ]:
pred = model.predict(x_test)
pred = pred[:,1] # Only positive cases
plot_roc(pred,y_compare)